In [1]:
import pyspark

In [2]:
pyspark.__version__

'3.4.1'

In [3]:
pyspark.__file__

'/home/ramcharansatyasaitejapolisetti/anaconda3/lib/python3.11/site-packages/pyspark/__init__.py'

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder\
                    .master("local[*]")\
                    .appName("test")\
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/23 03:41:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-10.parquet

--2024-01-23 03:41:06--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.85.39.97, 52.85.39.117, 52.85.39.65, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.39.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59009059 (56M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2023-10.parquet.3’

yellow_tripdata_202 100%[===================>]  56.28M  96.0MB/s    in 0.6s    

2024-01-23 03:41:07 (96.0 MB/s) - ‘yellow_tripdata_2023-10.parquet.3’ saved [59009059/59009059]



In [7]:
data = spark.read.parquet("yellow_tripdata_2023-10.parquet")

In [8]:
data.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2023-10-01 00:16:44|  2023-10-01 00:16:49|              1|          0.0|         1|                 N|         168|         168|           2|        3.0|  1.0|    0.5|       0.

In [9]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2024-01-23 03:41:13--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.130.72, 52.216.168.213, 52.217.43.174, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.130.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv.3’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2024-01-23 03:41:13 (104 MB/s) - ‘taxi+_zone_lookup.csv.3’ saved [12322/12322]



In [10]:
!head taxi+_zone_lookup.csv

"LocationID","Borough","Zone","service_zone"
1,"EWR","Newark Airport","EWR"
2,"Queens","Jamaica Bay","Boro Zone"
3,"Bronx","Allerton/Pelham Gardens","Boro Zone"
4,"Manhattan","Alphabet City","Yellow Zone"
5,"Staten Island","Arden Heights","Boro Zone"
6,"Staten Island","Arrochar/Fort Wadsworth","Boro Zone"
7,"Queens","Astoria","Boro Zone"
8,"Queens","Astoria Park","Boro Zone"
9,"Queens","Auburndale","Boro Zone"


In [11]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [12]:
df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [13]:
df.write.mode('overwrite').parquet('zones')

In [14]:
spark.stop()